## EVO-Prompt Sentiment Classification (Portuguese - imdb_pt)
### Etapas: População inicial, avaliação, evolução, repetição


### Configuração do Ambiente

In [79]:
%pip install datasets
%pip install openai==0.28.0
%pip install maritalk
%pip install numpy
%pip install scikit-learn
%pip install scipy
%pip install seaborn
%pip install matplotlib
%pip install pandas
%pip install sklearn
%pip install random
%pip install requests
%pip install logging
%pip install time

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
   

In [80]:
# Setup Inicial
import random
import os
import pandas as pd
import numpy as np
import requests
import json
import openai
import yaml
from sklearn.metrics import accuracy_score, f1_score

## Configuração de APIs (endpoint e token)

In [81]:
# Carregando configurações
creds_path = os.path.expanduser("~/repos/emo-prompt-project/config/credentials.yaml")
settings_path = os.path.expanduser("~/repos/emo-prompt-project/config/experiment_settings.yaml")

with open(creds_path) as f:
    creds = yaml.safe_load(f)
with open(settings_path) as f:
    settings = yaml.safe_load(f)

In [82]:
openai.api_key = creds["openai_api_key"]
openai.api_base = creds["openai_api_base"]

In [83]:
sabia_api_key = creds["sabia_api_key"]
sabia_url = creds["sabia_url"]
sabia_headers = {
    "Authorization": f"Bearer {sabia_api_key}",
    "Content-Type": "application/json"
}

## Preparação do Dataset

In [84]:
# Carregamento do Dataset (subconjunto local)
df = pd.read_csv("~/repos/emo-prompt-project/dataset/imdb_pt_subset1.csv")
df = df.dropna(subset=["text", "label"])
df = df[df["label"].isin([0, 1])]
texts = df["text"].tolist()
labels = df["label"].tolist()

In [85]:
# População Inicial de Prompts 
initial_prompts = [
    "Avalie o sentimento desta crítica de filme e classifique-a como 'positiva', 'negativa' ou 'neutra'.",
    "Determine se a opinião expressa na crítica a seguir reflete um sentimento positivo ou negativo sobre o filme.",
    "Classifique o tom geral desta análise de filme como sendo 'positivo', 'negativo' ou 'neutro'.",
    "Decida se a crítica abaixo expressa uma visão favorável ou desfavorável do filme.",
    "Analise o texto da crítica e determine se ela transmite um sentimento de aprovação ou desaprovação.",
    "Baseado na emoção expressa, indique se a seguinte resenha de filme é positiva, negativa ou neutra.",
    "Classifique a avaliação do filme como sendo predominantemente 'positiva', 'negativa' ou 'mista'.",
    "Determine a polaridade da crítica do filme: a opinião expressa é positiva ou negativa?",
    "Interprete o sentimento desta crítica e indique se o autor está elogiando ou criticando o filme.",
    "Analise o conteúdo emocional da crítica e classifique-a como 'favorável', 'desfavorável' ou 'imparcial'.",
]

In [86]:
# Parâmetros do experimento
dataset_path = settings["dataset_path"]
results_log_path = settings["results_log_path"]
models = settings["models"]
strategies = settings["strategies"]
top_k = settings["top_k"]

In [87]:
# Estratégias de prompting
def build_prompt(prompt_instruction, text, strategy):
    if strategy == "zero-shot":
        return f"{prompt_instruction}\n\nTexto: {text}\n\nResponda apenas com 0 (negativo) ou 1 (positivo)."
    elif strategy == "few-shot":
        examples = (
            "Texto: Esse filme é excelente, adorei cada momento.\nResposta: 1\n"
            "Texto: Que decepção! Perdi meu tempo assistindo.\nResposta: 0\n"
        )
        return f"{prompt_instruction}\n\n{examples}Texto: {text}\nResposta:"
    elif strategy == "cot":
        return (
            f"{prompt_instruction}\n\nTexto: {text}\n"
            f"Explique seu raciocínio antes de responder com 0 (negativo) ou 1 (positivo).\nResposta:"
        )


In [88]:
# GPT-4o Mini como gerador de prompts
def generate_prompt_with_gpt4o(top_prompts_with_scores):
    prompt_text = "\n".join([f"{i+1}. {p} (F1: {s:.2f})" for i, (p, s) in enumerate(top_prompts_with_scores)])
    system_instruction = "Você é um otimizador de prompts para classificação de sentimentos."
    user_instruction = (
        "Com base nos prompts abaixo e seus desempenhos, gere um novo prompt em português para a tarefa de "
        "classificação de sentimento (positivo ou negativo). Gere apenas o prompt:"
    )

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_instruction},
            {"role": "user", "content": f"{user_instruction}\n\n{prompt_text}"}
        ],
        temperature=0.7,
    )
    return response.choices[0].message["content"].strip()

In [89]:
def query_maritalk(prompt_instruction, text):
    request_data = {
        "model": "sabiazinho-3",
        "messages": [{"role": "user", "content": f"{prompt_instruction}\n\nTexto: {text}"}]
    }

    try:
        response = requests.post(
            url=creds["sabia_url"],
            headers={
                "Authorization": f"Bearer {creds['sabia_api_key']}",
                "Content-Type": "application/json"
            },
            json=request_data,
            timeout=10
        )
        response.raise_for_status()
        prediction = response.json().get("answer", "").strip().lower()
        return prediction
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição para Maritalk: {e}")
        return "erro"


In [90]:
# Avaliação de um prompt
def evaluate_prompt(prompt_instruction, model, strategy):
    preds = []
    gold = df["label"].tolist()

    for text, true_label in zip(df["text"], gold):
        prompt = build_prompt(prompt_instruction, text, strategy)

        if model == "maritalk":
            response = query_maritalk(prompt_instruction, text)
        else:
            response = "0"  # mock para outros modelos

        if "1" in response:
            prediction = 1
        elif "0" in response:
            prediction = 0
        else:
            prediction = 1 - true_label
            print(f"Resposta inesperada: '{response}'")

        preds.append(prediction)

    return accuracy_score(gold, preds), f1_score(gold, preds)

In [91]:
# Execução de um cenário
def run_scenario(model, strategy, initial_prompts):
    results = []

    # Avalia prompts iniciais
    for prompt in initial_prompts:
        acc, f1 = evaluate_prompt(prompt, model, strategy)
        results.append((prompt, acc, f1))

    results.sort(key=lambda x: x[2], reverse=True)
    top_prompts = results[:top_k]

    # Salva resultado parcial inicial
    os.makedirs("logs", exist_ok=True)
    log_file = f"logs/results_{model}_{strategy}.csv"
    pd.DataFrame(results, columns=["prompt", "accuracy", "f1"]).to_csv(log_file, index=False)

    # Tenta gerar novos prompts com GPT-4o Mini
    try:
        new_prompts = [generate_prompt_with_gpt4o([(p, f1)]) for p, _, f1 in top_prompts]
    except Exception as e:
        print(f"Erro ao gerar prompts com GPT-4o Mini: {e}")
        new_prompts = []

    # Avalia novos prompts
    for prompt in new_prompts:
        acc, f1 = evaluate_prompt(prompt, model, strategy)
        results.append((prompt, acc, f1))

    # Salva resultados finais
    pd.DataFrame(results, columns=["prompt", "accuracy", "f1"]).to_csv(log_file, index=False)
    return results


In [92]:
# Executa os 9 cenários
final_results = []

for model in models:
    for strategy in strategies:
        print(f"Executando: {model} + {strategy}")
        scenario_results = run_scenario(model, strategy, initial_prompts)
        for prompt, acc, f1 in scenario_results:
            final_results.append({
                "model": model,
                "strategy": strategy,
                "prompt": prompt,
                "accuracy": acc,
                "f1": f1,
            })

# Salvamento geral
pd.DataFrame(final_results).to_csv(results_log_path, index=False)


Executando: maritalk + zero-shot
Resposta inesperada: 'o sentimento desta crítica de filme é 'negativa'. a crítica contém várias expressões de descontentamento e decepção com o filme, destacando aspectos como os efeitos especiais (descritos como "roubo"), a história (questionada se realmente existiu), e a atuação (comparada de forma desfavorável a uma boneca). palavras como "decepcionado", "ridículo", "bagunça chata", "estúpido", "pior filme de terror", "má atriz", e "estúpido" reforçam a avaliação negativa. o tom geral é de desapontamento e descontentamento com o filme, classificando-o como uma má escolha tanto para os espectadores quanto para os críticos.'
Resposta inesperada: 'esta crítica de filme pode ser classificada como 'negativa'. o texto expressa claramente desapontamento e insatisfação com a série, utilizando termos como "perda de tempo e dinheiro", "embaraçoso" e "horrível". além disso, compara negativamente a adaptação com o material original, o livro, e sugere que a série

/home/sara/repos/emo-prompt-project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sara/repos/emo-prompt-project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sara/repos/emo-prompt-project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

Erro ao gerar prompts com GPT-4o Mini: Project `proj_PY18Vj5USyoF6IAj2IPID8Fi` does not have access to model `gpt-4o`
Executando: deepseek + few-shot


/home/sara/repos/emo-prompt-project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sara/repos/emo-prompt-project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sara/repos/emo-prompt-project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

Erro ao gerar prompts com GPT-4o Mini: Project `proj_PY18Vj5USyoF6IAj2IPID8Fi` does not have access to model `gpt-4o`
Executando: deepseek + cot


...

Erro ao gerar prompts com GPT-4o Mini: Project `proj_PY18Vj5USyoF6IAj2IPID8Fi` does not have access to model `gpt-4o`
Executando: llama + few-shot


/home/sara/repos/emo-prompt-project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sara/repos/emo-prompt-project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sara/repos/emo-prompt-project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

Erro ao gerar prompts com GPT-4o Mini: Project `proj_PY18Vj5USyoF6IAj2IPID8Fi` does not have access to model `gpt-4o`
Executando: llama + cot


/home/sara/repos/emo-prompt-project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sara/repos/emo-prompt-project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sara/repos/emo-prompt-project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

Erro ao gerar prompts com GPT-4o Mini: Project `proj_PY18Vj5USyoF6IAj2IPID8Fi` does not have access to model `gpt-4o`


## Funções Auxiliares

## Avaliação Final

In [ ]:
# Avaliação final
best_prompt = population[0]
final_accuracy = sum(
    evaluate_prompt(example['text'], best_prompt, example['label'], url, headers)
    for example in test_sample
) / len(test_sample)
logging.info(f"Melhor prompt: {best_prompt}")
logging.info(f"Acurácia final no conjunto de teste: {final_accuracy:.2%}")
print(f"Melhor prompt: {best_prompt}")
print(f"Acurácia final no conjunto de teste: {final_accuracy:.2%}")